In [2]:
import pandas as pd
import numpy as np
import ezodf

def read_ods_by_name(file_path, sheet_name):
    ods = ezodf.opendoc(file_path)
    
    sheet = ods.sheets[sheet_name]
    
    data = []
    for row in sheet.rows():
        data_row = [cell.value for cell in row]
        data.append(data_row)
    
    df = pd.DataFrame(data)
    return df


Reading the TM'S

In [3]:
file_path = 'SPORT_documents//sport_ttc_20220814.ods'

sheets = ['TM_CDH_1', 'TM_CDH_2', 'TM_PLD', 'TM_DSU', 'TM_ACS', 'TM_SCI', 'TM_HOU', 'TM_STD', 'TM_BEA']

columns = ['identification','name','version_number', 'pkt_type', 'sec_hdr_flag', 'apid', 'seq_flags', 'seq_count', 'pkt_data_length', 'secondary_header', 'data', 'checksum']

main_tm_df = pd.DataFrame()

for sheet_name in sheets:
    df = read_ods_by_name(file_path, sheet_name)
    df = df.iloc[6:, 0:12].dropna(axis = 0, how = 'all')

    main_tm_df = pd.concat([main_tm_df, df])

main_tm_df.columns = columns

main_tm_df['apid'] = main_tm_df['apid'].apply(lambda x: hex(int(x, 16)))

main_tm_df[['identification','apid', 'data']]

,identification,apid,data
6,TM_CDH_001,0x201,DD_CDH_001
7,TM_CDH_005,0x205,DD_CDH_005
8,TM_CDH_006,0x206,DD_CDH_006
9,TM_CDH_011,0x20b,DD_CDH_011
10,TM_CDH_012,0x20c,DD_CDH_012
...,...,...,...
49,TM_HOU_021_C,0x3d,DC_HOU_OBS_4E
50,TM_HOU_022_C,0x3e,DC_HOU_OBS_5E
51,TM_HOU_023_C,0x3f,DC_HOU_OBS_6E
6,TM_STD_001,0x1,Varies


Reading the DD's

In [4]:
def single_data_field_dict(
        field_name: str, 
        length_bits: int, 
        format_str: str, 
        nominal_min: int| float | None, 
        nominal_max: int| float | None, 
        conversion: str | None, 
        unit: str | None,
        observation = None
    ) -> dict:

    single_data_dict = {
        "field": field_name,
        "lenght(bits)": length_bits,
        "format": format_str,
        "nominal_minimum": nominal_min,
        "nominal_maximum": nominal_max,
        "conversion": conversion,
        "unit": unit,
        "observation": observation
    }

    return single_data_dict

In [5]:
file_path = 'SPORT_documents//sport_ttc_20220814.ods'
sheet_name = 'DD_HOU'  
df = read_ods_by_name(file_path, sheet_name)

df = df.dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all')

data_array = df.to_numpy()

def read_data_dict_from_excel_format(data_array: np.ndarray):
    
    assert data_array[0,0] is not None
    assert data_array[1,0] == 'Field'
    assert data_array[-1,0] == 'Total'
    
    main_dict = {
        "data_name": data_array[0,0]
    }

    first_row_to_read = data_array[2,1:]
    all_values_are_none = np.all(first_row_to_read == None)

    # if all_values_are_none:
    #     data_packets = None
    # else:
    data_packets = []
    for i in range(2, len(data_array)):
        row = data_array[i,:]

        field_name, bit_length, format_str, nominal_min, nominal_max, conversion, unit, observation = row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7]
        
        data_packets.append(single_data_field_dict(field_name, bit_length, format_str, nominal_min, nominal_max, conversion, unit, observation))

    main_dict["data_packets"] = data_packets

    return main_dict


pointer = 0
for i in range(0, len(data_array)):
    first_column_value = data_array[i,0]

    if first_column_value == 'Total':
        
        inner_data_array = data_array[pointer:(i+1), :]
        print('found total')
        # print(inner_data_array)
        print(read_data_dict_from_excel_format(inner_data_array))
        pointer = (i+1)
        print('===========')

found total
{'data_name': 'DD_HOU_CDH', 'data_packets': [{'field': 'GPS time (C&DH view)', 'lenght(bits)': 64.0, 'format': 'GPS time', 'nominal_minimum': 'N/A', 'nominal_maximum': 'N/A', 'conversion': 'N/A', 'unit': 'N/A', 'observation': None}, {'field': 'C&DH free-running clock', 'lenght(bits)': 64.0, 'format': 'GPS time', 'nominal_minimum': 'N/A', 'nominal_maximum': 'N/A', 'conversion': 'N/A', 'unit': 'N/A', 'observation': None}, {'field': 'GPS time origin', 'lenght(bits)': 8.0, 'format': 'uint8', 'nominal_minimum': 'N/A', 'nominal_maximum': 'N/A', 'conversion': 'N/A', 'unit': 'N/A', 'observation': '0 is FRC, 1 is GROUND, 2 is CTECS, 3 is CTECS EXTRAP'}, {'field': 'Number of boots', 'lenght(bits)': 32.0, 'format': 'uint32', 'nominal_minimum': 'N/A', 'nominal_maximum': 'N/A', 'conversion': 'N/A', 'unit': 'N/A', 'observation': None}, {'field': 'Number of valid uplink frames received', 'lenght(bits)': 32.0, 'format': 'uint32', 'nominal_minimum': 'N/A', 'nominal_maximum': 'N/A', 'convers

In [6]:
data_array[:,0]

array(['DD_HOU_CDH', 'Field', 'GPS time (C&DH view)',
       'C&DH free-running clock', 'GPS time origin', 'Number of boots',
       'Number of valid uplink frames received',
       'Number of telecommands in schedule', 'Current operating mode',
       'Free-running clock on last PPS', 'Power monitor enabled',
       'SCIENCE scheduler enabled', 'Total', 'DC_HOU_CDH', 'Field',
       'Compressed DD_HOU_CDH', 'Total', 'DD_HOU_EPS_1', 'Field',
       'BCR output current', 'BCR output voltage',
       '3.3V current draw of EPS', '5V current draw of EPS',
       'Output current of 12V bus', 'Output voltage of 12V bus',
       'Output current of VBAT bus', 'Output voltage of VBAT bus',
       'Output current of 5V bus', 'Output voltage of 5V bus',
       'Output current of 3.3V bus', 'Output voltage of 3.3V bus',
       'Output current of switch 1', 'Output voltage of switch 1',
       'Output current of switch 2', 'Output voltage of switch 2',
       'Output current of switch 3', 'Output v